# **Step 1: Imports & Setup**

In [38]:
import os
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

np.random.seed(42)
tf.random.set_seed(42)

# **Step 2: Dataset Generation**

In [39]:
# Parameters
IMG_SIZE = 50
OUTPUT_DIR = "dataset"
IMG_DIR = os.path.join(OUTPUT_DIR, "images")
LABEL_FILE = os.path.join(OUTPUT_DIR, "labels.csv")

os.makedirs(IMG_DIR, exist_ok=True)

labels = []
img_id = 0

for y in range(IMG_SIZE):
    for x in range(IMG_SIZE):
        img = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.uint8)
        img[y, x] = 255

        img_name = f"img_{img_id:04d}.png"
        img_path = os.path.join(IMG_DIR, img_name)
        cv2.imwrite(img_path, img)

        labels.append([img_name, x, y])
        img_id += 1

with open(LABEL_FILE, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["image_name", "x", "y"])
    writer.writerows(labels)

print(f"Dataset created with {img_id} images.")

Dataset created with 2500 images.


# **Step 3: Load Dataset**

In [41]:
def load_dataset(img_dir, label_file):
    images = []
    heatmaps = []

    with open(label_file, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            img_path = os.path.join(img_dir, row["image_name"])
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            img = img.astype(np.float32) / 255.0
            img = np.expand_dims(img, axis=-1)

            heatmap = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)
            heatmap[int(row["y"]), int(row["x"])] = 1.0
            heatmap = np.expand_dims(heatmap, axis=-1)

            images.append(img)
            heatmaps.append(heatmap)

    return np.array(images), np.array(heatmaps)


X, Y = load_dataset(IMG_DIR, LABEL_FILE)

print("Images shape:", X.shape)
print("Labels shape:", Y.shape)

Images shape: (2500, 50, 50, 1)
Labels shape: (2500, 50, 50, 1)


# **Step 4: Train / Val / Test Split**

In [42]:
X_train, X_temp, Y_train, Y_temp = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp, Y_temp, test_size=0.5, random_state=42
)

print("Train:", X_train.shape)
print("Val:", X_val.shape)
print("Test:", X_test.shape)

Train: (2000, 50, 50, 1)
Val: (250, 50, 50, 1)
Test: (250, 50, 50, 1)


# **Step 5: Model Definition**

In [43]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(50, 50, 1)),
    tf.keras.layers.Conv2D(16, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.Conv2D(1, (1, 1), padding="same", activation="linear")
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 50, 50, 16)        160       
                                                                 
 conv2d_10 (Conv2D)          (None, 50, 50, 32)        4640      
                                                                 
 conv2d_11 (Conv2D)          (None, 50, 50, 1)         33        
                                                                 
Total params: 4833 (18.88 KB)
Trainable params: 4833 (18.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# **Step 6: Compile & Train**

In [44]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    X_train,
    Y_train,
    validation_data=(X_val, Y_val),
    epochs=15,
    batch_size=32
)

Epoch 1/15
63/63 [==============================] - 2s 26ms/step - loss: 6.2677e-04 - accuracy: 0.9996 - val_loss: 2.9672e-04 - val_accuracy: 0.9996
Epoch 2/15
63/63 [==============================] - 2s 26ms/step - loss: 1.3011e-04 - accuracy: 1.0000 - val_loss: 1.5361e-05 - val_accuracy: 1.0000
Epoch 3/15
63/63 [==============================] - 2s 24ms/step - loss: 1.0838e-05 - accuracy: 1.0000 - val_loss: 1.1988e-05 - val_accuracy: 1.0000
Epoch 4/15
63/63 [==============================] - 2s 25ms/step - loss: 8.6685e-06 - accuracy: 1.0000 - val_loss: 9.9105e-06 - val_accuracy: 1.0000
Epoch 5/15
63/63 [==============================] - 2s 24ms/step - loss: 7.1824e-06 - accuracy: 1.0000 - val_loss: 7.6933e-06 - val_accuracy: 1.0000
Epoch 6/15
63/63 [==============================] - 1s 23ms/step - loss: 5.6031e-06 - accuracy: 1.0000 - val_loss: 5.5858e-06 - val_accuracy: 1.0000
Epoch 7/15
63/63 [==============================] - 1s 22ms/step - loss: 4.2507e-06 - accuracy: 1.0000 - v

# **Step 7: Save Training Graphs (Loss + Accuracy)**

In [45]:
os.makedirs("results", exist_ok=True)

# Loss plot
plt.figure()
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Training vs Validation Loss")
plt.savefig("results/loss_curve.png")
plt.close()

# Accuracy plot
plt.figure()
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Training vs Validation Accuracy")
plt.savefig("results/accuracy_curve.png")
plt.close()

print("Training graphs saved.")


Training graphs saved.


# **Step 8: Evaluation on Test Set**

In [46]:
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Test Loss: {test_loss:.6f}")
print(f"Test Accuracy: {test_acc:.6f}")

8/8 [==============================] - 0s 7ms/step - loss: 7.4696e-07 - accuracy: 1.0000
Test Loss: 0.000001
Test Accuracy: 1.000000


# **Step 9: Coordinate Extraction (Argmax)**

In [47]:
def extract_coordinates(heatmap):
    idx = np.argmax(heatmap)
    y, x = np.unravel_index(idx, (IMG_SIZE, IMG_SIZE))
    return x, y


predictions = model.predict(X_test)

gt_coords = []
pred_coords = []

for i in range(len(X_test)):
    gt_coords.append(extract_coordinates(Y_test[i]))
    pred_coords.append(extract_coordinates(predictions[i]))

8/8 [==============================] - 0s 6ms/step


# **Step 10: Ground Truth vs Prediction Visualization**

In [48]:
for i in range(5):
    img = X_test[i].squeeze()
    gt_x, gt_y = gt_coords[i]
    pr_x, pr_y = pred_coords[i]

    plt.imshow(img, cmap="gray")
    plt.scatter(gt_x, gt_y, c="green", label="Ground Truth")
    plt.scatter(pr_x, pr_y, c="red", marker="x", label="Prediction")
    plt.legend()
    plt.title(f"GT: ({gt_x},{gt_y}) | Pred: ({pr_x},{pr_y})")
    plt.savefig(f"results/sample_{i}.png")
    plt.close()

print("Sample predictions saved.")

Sample predictions saved.


# **Step 11: Save Model**

In [49]:
model.save("pixel_locator_model.h5")
print("Model saved successfully.")

Model saved successfully.


/Users/deveshattri/Desktop/Internship Tasks/Deepedge/.venv/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
